In [ ]:
import warnings
warnings.filterwarnings("ignore")
import sys
sys.path.insert(1, '/mnt/c/MBA/')
from model_functions import *
import numpy as np
import pandas as pd
from datetime import datetime,date, timedelta
import statsmodels.api as sm
import matplotlib.pyplot as plt
from prophet import Prophet  
from prophet.diagnostics import cross_validation
from prophet.serialize import model_to_json, model_from_json
from statsmodels.tsa.statespace.sarimax import SARIMAX
import json
from dateutil.relativedelta import relativedelta
import os
import glob


In [ ]:
root = '/mnt/c/MBA'

list_files = glob.glob(f'{root}/data2/*.parquet*') 

i= 0

print(len(list_files))

for file in list_files:

    print(i)

    frame= pd.read_parquet(file, columns=['date_ref', 'WS100'])

    df=frame[(frame.groupby('date_ref').cumcount()== 4)].reset_index(drop=True)

    i+=1

In [ ]:

df=df.drop_duplicates().reset_index(drop=True)

df['date_ref'] = pd.to_datetime(df['date_ref'])

df = df.drop(df.tail(11).index)

df = df.sort_values('date_ref').reset_index(drop=True)

df = df.set_index('date_ref').resample('MS').mean().reset_index()

df = df.rename({'date_ref': 'ds', 'WS100': 'y'}, axis=1)

df = df.set_index('ds').asfreq('MS')

df =add_fourier_terms(df, 1)

df.to_parquet(f'{root}/wind.parquet', engine='fastparquet')

dataframe = pd.read_parquet(f'{root}/wind.parquet', engine='fastparquet')


In [ ]:


directory=os.path.join(root,'figures')
    
if not os.path.exists(directory):
  os.makedirs(directory)

directory=os.path.join(root,'models')
    
if not os.path.exists(directory):
  os.makedirs(directory)

In [ ]:

period = 12

orders = {}

auto_arima_dict = {'serie': None, 'start_p': 1, 'start_q': 1,
            'start_P': 1, 'start_Q': 1, 
            'max_p': 2, 'max_q': 2,
            'max_P': 5, 'max_Q': 5,
            'max_d': 2, 'max_D': 1,
            'max_order': None,  'd': None, 'D': None,
            'test': "adf", 'm': 12,
            'stepwise': True, 'trace': True,
            'stationary': False, 'seasonal': True}


serie = dataframe[['y']]

trend_estimate_ad, periodic_estimate_ad, residual_ad = decomp(serie,period, root)

check_stationarity_plots_acf_pacf(serie, trend_estimate_ad, periodic_estimate_ad, residual_ad, root)

auto_arima_dict['serie'] = serie

order = sarimax_diagnostic(auto_arima_dict, root)

orders.update(order)

dataframe = dataframe.reset_index()


In [ ]:

json_string = json.dumps(orders)

with open(f'{root}/json_data.json', 'w') as outfile:
    outfile.write(json_string)

with open(f'{root}/json_data.json') as json_file:
    orders = json.load(json_file)


In [ ]:

grid_optuna='optuna'

tuning_results_all = pd.DataFrame()

initial= 12 * 30
period= 1 

horizon_sarimax = 4
horizon_prophet = f'{3 * 30 +10} days'

run_cross_validation = True

exog_cols=dataframe.reset_index(drop=True).drop(['ds', 'y'], axis=1).columns.to_list()

In [ ]:

if run_cross_validation==True:
  
    sarimax_cv_all = sarimax_crosvalidation(df=dataframe, order=orders['orders'][0], seasonal_order=orders['orders'][1], initial=initial, period=period, 
                                        horizon=horizon_sarimax, exog_col=exog_cols).sort_values('ds').reset_index(drop=True)

    m = Prophet(interval_width=0.95)
    
    if exog_cols != None:
        for col in exog_cols:
            m.add_regressor(col)   

    m.fit(dataframe)

    plots_prophet(m, sarimax_cv_all, prefix='Sarimax', root=root)

    if os.path.isfile('models/Prophet_bst_params.json'):
        with open(f'{root}/models/Prophet_bst_params.json', 'r') as x:
            best_params = json.load(x)

    elif grid_optuna == 'grid':          
        best_params, tuning_results = grid_search_prophet(df=dataframe, initial=initial, period=period, horizon=horizon_prophet, param_grid={}, exog_col=exog_cols)
        tuning_results_all = tuning_results_all.append(tuning_results)
        with open(f'{root}/models/Prophet_bst_params.json', 'w') as x:
            json.dump(best_params, x)

    elif grid_optuna =='optuna':
        best_params =optuna_prophet(df=dataframe, initial=initial, period=period, horizon=horizon_prophet)
        with open(f'{root}/models/Prophet_bst_params.json', 'w') as x:
            json.dump(best_params, x)
    else:
        best_params = {'changepoint_prior_scale': 0.1,
            'seasonality_prior_scale': 0.1,
            'seasonality_mode': 'multiplicative',
            'weekly_seasonality':False,
                }
            
    m = Prophet(**best_params, interval_width=0.95)

    if exog_cols != None:
        for col in exog_cols:
            m.add_regressor(col)     

    m.fit(dataframe)  

    cutoffs = []
    cutoff  = dataframe.ds.min()+relativedelta(months=initial)
    while cutoff<dataframe.ds.max()-relativedelta(months=period*5):
        cutoffs=cutoffs + [cutoff]
        cutoff = cutoff+relativedelta(months=period)

    prophet_cv_all = cross_validation(m, cutoffs=cutoffs, horizon=horizon_prophet).sort_values('ds').reset_index(drop=True)

    with open(f'{root}/models/prophet_model', 'w') as fout:
        fout.write(model_to_json(m))       
    with open(f'{root}/models/prophet_model', 'r') as fin:
        m = model_from_json(fin.read())

    plots_prophet(m, prophet_cv_all, prefix='Prophet', root=root)



In [ ]:
future_dates = pd.date_range(start=dataframe['ds'].max()+pd.offsets.MonthBegin(1), periods=3, freq='MS')

# SARIMAX
#--------
if exog_cols != None:
    model = SARIMAX(dataframe[['y']], order=orders['orders'][0], seasonal_order=orders['orders'][1], exog=dataframe[exog_cols],enforce_stationarity=False)

else:
    model = SARIMAX(dataframe[['y']], order=orders['orders'][0], seasonal_order=orders['orders'][1],enforce_stationarity=False)

sarimax_model = model.fit(disp=0)

 
sarimax_future = pd.DataFrame({'ds': future_dates})
sarimax_future = sarimax_future.set_index('ds').asfreq('MS')
sarimax_future = add_fourier_terms(sarimax_future, 1).reset_index()

if exog_cols != None:

    forecast = sarimax_model.get_forecast(steps=3, alpha=0.05, exog=sarimax_future[exog_cols])

else:
    
    forecast = sarimax_model.get_forecast(steps=3, alpha=0.05)

sarimax_future['yhat']       = forecast.predicted_mean.reset_index(drop=True)
sarimax_future['yhat_upper'] = forecast.conf_int()['upper y'].reset_index(drop=True)
sarimax_future['yhat_lower'] = forecast.conf_int()['lower y'].reset_index(drop=True)

if exog_cols != None:

    sarimax_future = sarimax_future[['ds', 'yhat', 'yhat_lower', 'yhat_upper']+ exog_cols]
    
else:
    
    sarimax_future = sarimax_future[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

sarimax_future = sarimax_future.dropna()

sarimax_future['cutoff']  = dataframe['ds'].max()

with open(f'{root}/models/Prophet_bst_params.json', 'r') as x:
    best_params = json.load(x)

m = Prophet(**best_params, interval_width=0.95)

if exog_cols != None:
    for col in exog_cols:
        m.add_regressor(col)   

m.fit(dataframe)  

prophet_future = pd.DataFrame({'ds': future_dates})
prophet_future = prophet_future.set_index('ds').asfreq('MS')
prophet_future =add_fourier_terms(prophet_future, 1).reset_index()

if exog_cols != None:

    prophet_future = m.predict(prophet_future)[['ds','yhat', 'yhat_lower', 'yhat_upper']+exog_cols]

else:
    
    prophet_future = m.predict(prophet_future)[['ds','yhat', 'yhat_lower', 'yhat_upper']]

prophet_future['cutoff'] = dataframe['ds'].max()


prophet_cv_all = pd.concat([prophet_cv_all,prophet_future], axis=0).reset_index(drop=True)
sarimax_cv_all = pd.concat([sarimax_cv_all,sarimax_future], axis=0).reset_index(drop=True)

In [ ]:
prophet_cv_all['month_sin'+str(1)] = prophet_cv_all.ds.apply(lambda x: np.sin(2 *1* np.pi * x.month/12)) 
prophet_cv_all['month_cos'+str(1)] = prophet_cv_all.ds.apply(lambda x:np.cos(2 *1* np.pi * x.month/12))

In [ ]:
if grid_optuna=='grid':
    tuning_results_all.to_excel('tuning_results.xlsx')

prophet_cv_all['date_ref'] = prophet_cv_all.cutoff + pd.offsets.MonthBegin(1)
sarimax_cv_all['date_ref'] = sarimax_cv_all.cutoff + pd.offsets.MonthBegin(1)

prophet_cv_all.to_excel(f'{root}/Prophet_CrossValidation.xlsx')
sarimax_cv_all.to_excel(f'{root}/Sarimax_CrossValidation.xlsx')


In [ ]:
prophet_cv_all = pd.read_excel(f'{root}/Prophet_CrossValidation.xlsx', index_col=0)
sarimax_cv_all = pd.read_excel(f'{root}/Sarimax_CrossValidation.xlsx', index_col=0)

In [ ]:

df  = prophet_cv_all.rename(columns={'ds':'date_forecast', 'yhat':'yhat_prophet'})


if exog_cols != None:

    if 'y' in df.columns:
        df  = df[['date_ref', 'date_forecast', 'yhat_prophet', 'y']+exog_cols]
    else:
        df = df[['date_ref', 'date_forecast',  'yhat_prophet']+exog_cols]
        df['y'] = np.nan

else:

    if 'y' in df.columns:
        df  = df[['date_ref', 'date_forecast', 'yhat_prophet', 'y']]
    else:
        df = df[['date_ref', 'date_forecast',  'yhat_prophet']]
        df['y'] = np.nan

df2 = sarimax_cv_all.rename(columns={'ds':'date_forecast', 'yhat':'yhat_sarimax'})

if exog_cols != None:

    if 'y' in df2.columns:
        df2  = df2[['date_ref', 'date_forecast', 'yhat_sarimax', 'y']+exog_cols]
    else:
        df2 = df2[['date_ref', 'date_forecast',  'yhat_sarimax']+exog_cols]
        df2['y'] = np.nan

else:

    if 'y' in df2.columns:
        df2  = df2[['date_ref', 'date_forecast', 'yhat_sarimax', 'y']]
    else:
        df2 = df2[['date_ref', 'date_forecast',  'yhat_sarimax']]
        df2['y'] = np.nan

df = df.merge(df2, how = 'left', on = ['date_ref', 'date_forecast', 'y']+exog_cols)

df['delta'] = np.round(((df['date_forecast']-df['date_ref'])/np.timedelta64(1, 'M'))).astype(int)

df['delta'] = df['delta']+1

df = df.loc[(~df.yhat_prophet.isnull()) & (~df.yhat_sarimax.isnull())]

In [ ]:
df = df[['date_ref', 'date_forecast', 'delta', 'yhat_prophet', 'yhat_sarimax', 'y']+exog_cols]

In [ ]:
features = ['yhat_prophet', 'yhat_sarimax']+exog_cols
ohe_features = ['delta']

df_reg = pd.DataFrame()

date_refs = df.sort_values('date_ref').date_ref.unique()
cutoffs = date_refs[round(len(date_refs)/2):]
if run_cross_validation==True:   
    for cutoff in cutoffs:
        df_reg = df_reg.append(train_pred(df, cutoff, features, ohe_features))
else:
    for cutoff in cutoffs[-4:]:
        df_reg = df_reg.append(train_pred(df, cutoff, features, ohe_features))

In [ ]:
df_reg = df_reg[['date_ref', 'date_forecast', 'delta', 'yhat_prophet', 'yhat_sarimax',  'yhat_rf', 'yhat_lr', 'yhat_ls', 'yhat_dt', 'yhat_gbr', 'yhat_ada', 'y']+exog_cols]  

print("Delete Values with date_ref that are being included:")
date_refs = [str(x)[0:10] for x in df_reg.date_ref.unique()]
date_refs_str = str(date_refs).replace('[', '(').replace(']', ')')

In [ ]:

if run_cross_validation==True:
    df_reg = df_reg.dropna()
    metricas = df_reg.groupby(['delta']).apply(metrics).reset_index()
    metricas.to_excel('metrica.xlsx')


In [ ]:
import matplotlib.pyplot as plt
    
fig= plt.figure(figsize=(8, 6))
plt.grid(False)
plt.gca().spines[['right', 'top']].set_visible(False)
#plt.title("RMSE by Day")
plt.plot(metricas.delta, metricas.rmse, color='darkred',label='Prophet')
plt.plot(metricas.delta, metricas.rmse_sarimax, color='orange',label='Sarimax')
plt.plot(metricas.delta, metricas.rmse_lr, color='lime', label='LinearRegression')
plt.plot(metricas.delta, metricas.rmse_rf, color='dodgerblue', label='RandomForestRegressor')
plt.plot(metricas.delta, metricas.rmse_ls, color='blue', label='Lasso')
plt.plot(metricas.delta, metricas.rmse_dt, color='darkviolet', label='DecisionTreeRegressor')
plt.plot(metricas.delta, metricas.rmse_gbr, color='gray', label='GradientBoostingRegressor')
plt.plot(metricas.delta, metricas.rmse_ada, color='black', label='AdaBoostRegressor')
plt.legend(loc='lower left', bbox_to_anchor=(-0.05, -0.30), ncol=3)
plt.xlabel('Meses à frente')
plt.ylabel('RMSE')
plt.xticks([x for x in np.arange(1,4)], [str(x) for x in np.arange(1,4)])
fig.savefig(f'{root}/figures/RMSE_by_month.png', bbox_inches='tight')    
plt.show()
#------
fig= plt.figure(figsize=(8, 6))
plt.grid(False)
plt.gca().spines[['right', 'top']].set_visible(False)
#plt.title("MAPE por Delta")

plt.plot(metricas.delta, metricas.mape, color='darkred',label='Prophet')
plt.plot(metricas.delta, metricas.mape_sarimax, color='orange',label='Sarimax')
plt.plot(metricas.delta, metricas.mape_lr, color='lime', label='LinearRegression')
plt.plot(metricas.delta, metricas.mape_rf, color='dodgerblue', label='RandomForestRegressor')
plt.plot(metricas.delta, metricas.mape_ls, color='blue', label='Lasso')
plt.plot(metricas.delta, metricas.mape_dt, color='darkviolet', label='DecisionTreeRegressor')
plt.plot(metricas.delta, metricas.mape_gbr, color='gray', label='GradientBoostingRegressor')
plt.plot(metricas.delta, metricas.mape_ada, color='black', label='AdaBoostRegressor')
plt.legend(loc='lower left', bbox_to_anchor=(-0.05, -0.30), ncol=3)
plt.xlabel('Meses à frente')
plt.ylabel('MAPE')
plt.xticks([x for x in np.arange(1,4)], [str(x) for x in np.arange(1,4)])
fig.savefig(f'{root}/figures/MAPE_by_month.png', bbox_inches='tight')
plt.show()
#------
fig= plt.figure(figsize=(8, 6))
plt.grid(False)
plt.gca().spines[['right', 'top']].set_visible(False)
#plt.title("R² by Day - {}")

plt.plot(metricas.delta, metricas.r2, color='darkred',label='Prophet')
plt.plot(metricas.delta, metricas.r2_sarimax, color='orange',label='Sarimax')
plt.plot(metricas.delta, metricas.r2_rf, color='dodgerblue', label='RandomForestRegressor')
plt.plot(metricas.delta, metricas.r2_lr, color='lime', label='LinearRegression')
plt.plot(metricas.delta, metricas.r2_ls, color='blue', label='Lasso')
plt.plot(metricas.delta, metricas.r2_dt, color='darkviolet', label='DecisionTreeRegressor')
plt.plot(metricas.delta, metricas.r2_gbr, color='gray', label='GradientBoostingRegressor')
plt.plot(metricas.delta, metricas.r2_ada, color='black', label='AdaBoostRegressor')
plt.legend(loc='lower left', bbox_to_anchor=(-0.05, -0.30), ncol=3)
plt.xlabel('Meses à frente')
plt.ylabel('R²')
plt.xticks([x for x in np.arange(1,4)], [str(x) for x in np.arange(1,4)])
fig.savefig(f'{root}/figures/R2_by_month.png', bbox_inches='tight')
plt.show()

In [ ]:
from matplotlib import rcParams

rcParams['axes.linewidth'] = 1.5

plt.figure(figsize=(8, 5))

print('Correlations')
print(df_reg[['yhat_prophet', 'yhat_sarimax',   'yhat_lr', 'yhat_rf','yhat_ls', 'yhat_dt', 'yhat_gbr', 'yhat_ada', 'y']].corr()['y'])
plots= df_reg[['yhat_prophet', 'yhat_sarimax',   'yhat_lr', 'yhat_rf','yhat_ls', 'yhat_dt', 'yhat_gbr', 'yhat_ada', 'y']].corr(method='pearson').abs().plot.bar(y='y', color=['darkred','orange','dodgerblue',
                                                                                                                                                                'lime','blue','darkviolet','gray',
                                                                                                                                                                  'black', 'yellow'])
for bar in plots.patches:

    plots.annotate(format(bar.get_height(), '.2f'),
                   (bar.get_x() + bar.get_width() / 2,
                    bar.get_height()), ha='center', va='center',
                   size=8, xytext=(0, 8),
                   textcoords='offset points')

plt.grid(False)
plt.gca().spines[['right', 'top']].set_visible(False)
plt.tight_layout()
plt.savefig(f'{root}/figures/corr_complete.png')
plt.show()